$$
\begin{aligned}
    g\left( k \right) &=  \frac{\sqrt{2} }{2} \cdot   \frac{\sqrt{2+  \sqrt{3}}  }{3} \frac{\sqrt{2 +  \sqrt{3 +  \sqrt{4} } } }{4} \cdot  \ldots \frac{\sqrt{2 +  \sqrt{3 +  \ldots +  \sqrt{k} } } }{k}
\end{aligned}
$$


In [12]:
from __future__ import division
from sympy import *
x, y, z, t = symbols('x y z t')
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
init_printing()
init_printing(use_latex='mathjax', latex_mode='equation')


import pyperclip
def lx(expr):
    pyperclip.copy(latex(expr))
    print(expr)

import numpy as np
import matplotlib as plt

First let's see if we can just print the numbers

In [15]:
expr = 0
k = 7
for i in reversed(range(k)):
    print(i)

6
5
4
3
2
1
0


Right now let's start building an expression to get each term of the sequence:

In [80]:
expr = 0
k = 1
for i in reversed(range(2, k+2)):
    expr = sqrt(i + expr)
expr/(k+1)

√2
──
2 

In [81]:
expr = 0
k = 2
for i in reversed(range(2, k+2)):
    expr = sqrt(i + expr)
expr/(k+1)

  ________
╲╱ √3 + 2 
──────────
    3     

In [82]:
expr = 0
k = 3
for i in reversed(range(2, k+2)):
    expr = sqrt(i + expr)
expr/(k+1)

  ________
╲╱ 2 + √5 
──────────
    4     

So it seems to work but it's decided to simplify the surd which will get confusing, so avoid that we will pass the `evaluate = False` paraemeter [as described in the docs](https://docs.sympy.org/latest/tutorial/manipulation.html#prevent-expression-evaluation)

In [83]:
expr = 0
k = 3
for i in reversed(range(2, k+2)):
    expr = sqrt(i + expr, evaluate=False)
expr/(k+1)

   ________________
  ╱       ________ 
╲╱  2 + ╲╱ √4 + 3  
───────────────────
         4         

Now let's wrap the term generator into a function:

In [122]:
k = 3
def surd_seq(k):
    expr = 0
    for i in reversed(range(2, k+2)):
        expr = sqrt(i + expr, evaluate=False)
    return expr/(k+1)
surd_seq(3)


   ________________
  ╱       ________ 
╲╱  2 + ╲╱ √4 + 3  
───────────────────
         4         

Ok so that seems right so now let's instead look at a lot of the terms using list comprehension:

In [93]:
[surd_seq(i) for i in range(1, 7)]

⎡                                                                             
⎢                                                                         ____
⎢                                         _________________________      ╱    
⎢                   ________________     ╱        ________________      ╱     
⎢      ________    ╱       ________     ╱        ╱   ________          ╱      
⎢√2  ╲╱ √3 + 2   ╲╱  2 + ╲╱ √4 + 3    ╲╱   2 + ╲╱  ╲╱ √5 + 4  + 3    ╲╱    2 +
⎢──, ──────────, ───────────────────, ─────────────────────────────, ─────────
⎣2       3                4                         5                         

                                       _______________________________________
_______________________________       ╱          _____________________________
     _________________________       ╱          ╱     ________________________
    ╱    ________________           ╱          ╱     ╱    ________________    
   ╱    ╱   ________               ╱          ╱    

Unfourtunately there is no simple way, that I am aware of, to have it print the terms in the order we would expect, which is unfourtunate.

Now that we have the sequence, we can turn it into a series with the prod function:

In [125]:
l = [surd_seq(i) for i in range(1,7)]
prod(l)

def surd_ser(k):
    k = k + 1 # OBOB
    l = [surd_seq(i) for i in range(1,k)]
    return prod(l)

surd_ser(2)/surd_seq(1)


  ________
╲╱ √3 + 2 
──────────
    3     

So now this seems to check out so let's go onto the actual problem and look at $\text{surd_ser} \left( 100 \right)$, unfourtunately we won't be able to use `simplify` here because each term relies on the previous term and hence this will not scale in polynomial time, rather it will scale in exponential time and it's going to be very resource intensive.

To get an idea of what I mean, compare the time to evaluate 7 and 14 terms:



In [141]:
simplify(surd_ser(7))

                                                                              
                                                                              
                                                 _____________________________
                  _________________________     ╱         ____________________
                 ╱        ________________     ╱         ╱    ________________
     ________   ╱        ╱   ________         ╱         ╱    ╱   ________     
√2⋅╲╱ 2 + √5 ⋅╲╱   2 + ╲╱  ╲╱ √5 + 4  + 3  ⋅╲╱    2 + ╲╱   ╲╱  ╲╱ √6 + 5  + 4 
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                   ___________
             ______________________________________________       ╱           
______      ╱          ___________________________________       ╱           ╱
_____      ╱          ╱     _______________________

In [142]:
simplify(surd_ser(14))

                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                                              
                                                 _____________________________
                  _________________________     ╱         ____________________
                 ╱        ________________     ╱         ╱    ________________
     ________   ╱        ╱   ________         ╱     

Instead we will need to approach this numerically, again though we will have performance limitations, consider for example, evaluating this expression to 40:

In [149]:
N(surd_ser(40))

1.18026611043774e-37

In [ ]:
Now consider evaluating it to 50:

In [151]:
N(surd_ser(50))

4.05422002157275e-51

It takes a considerably longer time, instead we need to look at this from a functional perspective and rewrite the function:

In [167]:
k = 3
def surd_seq(k, numerical = False):
    expr = 0
    if numerical:
        for i in reversed(range(2, k+2)):
            expr = N(sqrt(i + expr, evaluate=False))
    else:
        for i in reversed(range(2, k+2)):
            expr = sqrt(i + expr, evaluate=False)
    return expr/(k+1)
surd_seq(3)

def surd_ser(k, numerical = False):
    if numerical:
        k = k + 1 # OBOB
        l = [surd_seq(i, numerical = True) for i in range(1,k)]
    else:
        k = k + 1 # OBOB
        l = [surd_seq(i, numerical = False) for i in range(1,k)]
    return prod(l)

surd_ser(2, True)


0.455341801261480

Now we will be able to evaluate this much more easily:

In [170]:
surd_ser(100, numerical = True)

6.83824783929329e-129

In [ ]:
This clearly demonstrates that the value is less than $10^-100$

## Numerical Performance

Sympy does have some limitations, let's demonstrate this by pushing the evaluation to it's limits.

In [175]:
surd_ser(500, numerical = True)

1.28458730802734e-977

By rewriting this in `numpy` we can get even better performance, so first import `numpy`:

In [177]:
import numpy
np.sqrt(3)

1.7320508075688772

In [ ]:
Now it's just a matter of swapping `sqrt()` for `np.sqrt`.

In [180]:
k = 3
def surd_seq(k, numerical = False):
    expr = 0
    if numerical:
        for i in reversed(range(2, k+2)):
            expr = np.sqrt(i + expr, )
    else:
        for i in reversed(range(2, k+2)):
            expr = sqrt(i + expr, evaluate=False)
    return expr/(k+1)
surd_seq(3)

def surd_ser(k, numerical = False):
    if numerical:
        k = k + 1 # OBOB
        l = [surd_seq(i, numerical = True) for i in range(1,k)]
    else:
        k = k + 1 # OBOB
        l = [surd_seq(i, numerical = False) for i in range(1,k)]
    return prod(l)

surd_ser(2, True)


0.4553418012614796

In [188]:
surd_ser(500, numerical = True)

0.0

Now `0` doesn't seem helpful, but in an engineering context it is if you bear in mind that `numpy` uses `double` precision (as in double a `float32` which is `float64`), this has an accuracy of 16 digits.

So this answer tells us that this series reaches 0, to a `double` precision, by 500 iterations.

Symbolically this series is limited by 0.

## Choosing the Tool

What this example demonstrates is the need to choose the right tool in *Python*, compared to a language like *Mathematica*, *Python* requires the user to be mindful of what they are trying to do beforehand as opposed to after, generally:

| Library     | Use Case                                                           |
| ---         | ---                                                                |
| *SymPy*     | Necessary for Symbolic Algebra                                     |
| *SymPy* `N()` | When arbitrary numerical precision is required despite performance |
| *NumPy*     | When High Performance Numerical Solutions are required up to 15 dp |



## Series and Recursion

The problem we were dealing with provided this series, which we solved via a `for` loop:
$$
\begin{aligned}
    g\left( k \right) &=  \frac{\sqrt{2} }{2} \cdot   \frac{\sqrt{2+  \sqrt{3}}  }{3} \frac{\sqrt{2 +  \sqrt{3 +  \sqrt{4} } } }{4} \cdot  \ldots \frac{\sqrt{2 +  \sqrt{3 +  \ldots +  \sqrt{k} } } }{k}
\end{aligned}
$$

let's modify this for the sake of discussion:

$$
\begin{aligned}
h\left( k \right) = \frac{\sqrt{2}  }{2} \cdot  \frac{\sqrt{3 +  \sqrt{2} } }{3} \cdot  \frac{\sqrt{4 +  \sqrt{3 +  \sqrt{2} } } }{4} +  \ldots +  \frac{\sqrt{k +  \sqrt{k - 1 +  \ldots \sqrt{3 + \sqrt{2}  } } } }{k}
\end{aligned}
$$


Essentially this difference obviates the need to use the `reversed` function in the previous `for` loop.

The function $h$ can be expressed by the series:

$$
\begin{aligned}
h\left( k \right) = \prod^k_{i = 2} \left( \frac{f_k}{k}  \right)  \quad : \quad f_i = \sqrt{i +  f_{i - 1}}  
\end{aligned}
$$

Within *Python* this isn't actually to difficult to express, something to the effect of the following would be sufficient:

```{python}
from sympy import *
def h(k):
    if k > 2:
	return f(k) * f(k-1)
    else:
        return 1

def f(i):
    expr = 0
    if i > 2:
        return sqrt(i + f(i -1))
    else:
        return 1
```

This is a very natural way to define series and sequencies and can be contrasted with a for loop.



```{python}
from sympy import *
def h(k):
    k = k + 1 # OBOB
    l = [f(i) for i in range(1,k)]
    return prod(l)

def f(k):
    expr = 0
    for i in range(2, k+2):
        expr = sqrt(i + expr, evaluate=False)
    return expr/(k+1)
```


If a function can be defined using a loop it can always be defined via recursion as well, [^823] the question is which will be more appropriate, generally the process for determining which is more appropriate is to the effect of:

1. Write the problem in a way that is easier to write or is more appropriate for demonstration
2. If performance is necessary then consider restructuring recursion for loops
    - In languages such ***R*** and *Python* :snake:, loops are usually faster, although there may be exceptions to this.

### Some Functions are more difficult to express with Recursion in *Python*

Consider the function $g\left( k \right)$:

$$
\begin{aligned}
    g\left( k \right) &=  \frac{\sqrt{2} }{2} \cdot   \frac{\sqrt{2+  \sqrt{3}}  }{3} \frac{\sqrt{2 +  \sqrt{3 +  \sqrt{4} } } }{4} \cdot  \ldots \frac{\sqrt{2 +  \sqrt{3 +  \ldots +  \sqrt{k} } } }{k} \\
    &=  \prod^k_{i = 2} \left( \frac{f_k}{k}  \right) \quad : \quad f_{i} = \sqrt{i +  f_{i+1}}
\end{aligned}
$$

Observe that the sequence now *looks* forward, not back, to solve using a `for` loop the `reversed` function dealt with that issue quite nicely and the technique corresponding to $h\left( k \right)$ was able to be implemented, however mathematically this can be a little clumsy to illustrate, for example the following rewrite would allow this to occur:

$$
\begin{aligned}
    g\left( k \right) &=  \prod^k_{i = 2} \left( \frac{f_k}{k}  \right) \quad : \quad f_{i} = \sqrt{\left( k- i \right)  +  f_{k - i - 1}}
\end{aligned}
$$

Now the function could be performed recursively in *Python* in a similar way, in particular, by performing something like this:

```{python}
from sympy import *
def h(k):
    if k > 2:
	return f(k) * f(k-1)
    else:
        return 1

def f(i, k):
    if k > i:
        return 1

    if i > 2:
	return sqrt((k-i) + f(k - i -1))
    else:
	return 1
```

The thing that's worth noting though is that it is much tricker to implement the recursive approach for $g\left( k \right)$ than $h\left( k \right)$, whereac the approach with the loop doesn't really change but for the inclusion of the `reversed()` function.



```{python}
from sympy import *
def h(k):
    k = k + 1 # OBOB
    l = [f(i) for i in range(1,k)]
    return prod(l)

def f(k):
    expr = 0
    for i in reversed(range(2, k+2)):
        expr = sqrt(i + expr, evaluate=False)
    return expr/(k+1)
```




[^823]: [This Stack Answer puts it well](https://stackoverflow.com/a/2093703/12843551):
    > There's a simple ad hoc proof for this.  Since you can build a Turing complete language using strictly iterative structures and a Turning complete language using only recursive structures, then the two are therefore equivalent.
